# Create files mapping FACS well and plate to basic allele metadata

This script takes cherry picking and allele info files as input and produces a table relating FACS well and plate IDs to allele IDs and symbols. 

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os

In [2]:
dualipa_inputs = "../1_inputs"
dualipa_outputs = "../3_outputs"
allele_collection_inputs = "../../../1_allele_collection/1_inputs"
allele_collection_outputs = "../../../1_allele_collection/3_outputs"

Vidal lab ran experiment once and analyzed the data. Some alleles didn't pass the qa/qc thresholds or got no data. Batch5 is is a re-run for all alleles that failed in the first set of batches. Here, we process everything together from scratch. An allele should always be compared to the wt on the same batch and plate. The summary stats can then be compared across batches. 

In [3]:
# Read in cherry picking files for both the initial batches and the second round
cp_col = ['orf_id', 'mut_id', 'node_type','dest_pla_id', 'dest_pla', 'dest_well']

pdest_layout_initial_batches_df = pd.read_csv(f'{dualipa_inputs}/cherry_picking_files/summary_CP_VUSDUALXP.csv', index_col=0)[cp_col]
pdest_layout_b5_df = pd.read_csv(f'{dualipa_inputs}/cherry_picking_files/summary_CP_VUSDUALXP_batch5.csv', index_col=0)[cp_col]

pdest_layout_df = pd.concat([pdest_layout_b5_df, pdest_layout_initial_batches_df])
pdest_layout_df = pl.DataFrame(pdest_layout_df)
pdest_layout_df = pdest_layout_df.with_columns(
    pl.col("orf_id").cast(pl.Int64).alias("orf_id"),
    pl.col("mut_id").cast(pl.Int64).alias("mut_id")
)
pdest_layout_df
### This doesn't have well A01

orf_id,mut_id,node_type,dest_pla_id,dest_pla,dest_well
i64,i64,str,i64,str,str
56360,0,"""wt""",23,"""VUSDUALXP_23""","""A12"""
56360,0,"""wt""",23,"""VUSDUALXP_23""","""B12"""
56360,0,"""wt""",23,"""VUSDUALXP_23""","""C12"""
56360,0,"""wt""",23,"""VUSDUALXP_23""","""D12"""
9905,0,"""wt""",23,"""VUSDUALXP_23""","""F02"""
…,…,…,…,…,…
null,null,"""pdest_empty""",20,"""VUSDUALXP_20""","""A08"""
null,null,"""pdest_empty""",21,"""VUSDUALXP_21""","""A07"""
null,null,"""pdest_empty""",21,"""VUSDUALXP_21""","""A08"""


In [4]:
seq_confirm_code_df = pd.read_table(f"{allele_collection_inputs}/sequence_confirmation_class_code.tsv")
with pd.option_context('display.max_columns', None):
    display(seq_confirm_code_df)

,id,description
0,1,perfectly validated
1,2,"partially validated, >= 50% coverage"
2,3,wild type
3,4,"partial wild type, target not covered, >= 50% ..."
4,5,"target and off-target mutation, >= 50% coverage"
5,6,"off-target mutation, >= 50% coverage"
6,7,"truncated, < 50% coverage"
7,99,no reads


In [5]:
seq_confirm_res_df = pd.read_table(f"{allele_collection_outputs}/VarChampSeqConfirmationResult_Alleles_Pass_QC.tsv")
seq_confirm_res_df = seq_confirm_res_df.rename(
    {
        "orf_id_wt": "orf_id",
        "mutation_id_old": "mut_id"
    }, 
    axis=1
)
with pd.option_context('display.max_columns', None):
    display(seq_confirm_res_df)

seq_confirm_res_df = pl.DataFrame(seq_confirm_res_df)
display(seq_confirm_res_df)

,symbol,ensembl_gene_id,orf_id,mut_id,ccsb_mutation_id,ccsb_allele_id,spdi,nt_change,aa_change,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,mislocalization_plate,mislocalization_well,entry_sequenced,entry_sequence_confirmation_class,db_sequenced,db_sequence_confirmation_class,n2h_sequenced,n2h_sequence_confirmation_class,n2h_ref_sequence_confirmation_class,dualip_sequenced,dualip_sequence_confirmation_class,dualip_ref_sequence_confirmation_class,mislocalization_sequenced,mislocalization_sequence_confirmation_class,mislocalization_ref_sequence_confirmation_class,gene_variant,chr_num,nuc_loc,ref_allele,alt_allele,Chrom
0,GBA1,ENSG00000177628,2,6,CCSBVarC000001,ALE0000584,NC_000001.11:155240033:C:G,160G>C,Val54Leu,RC4,RC4_Mut_GDEh1026,H01,GDEhDisVCh_40054,F12,2.0,RC4_Mut_GDDh1026,H01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,GBA1_Val54Leu,1,155240033,C,G,1
1,GBA1,ENSG00000177628,2,73,CCSBVarC000002,ALE00000002,NC_000001.11:155238225:G:A,670C>T,Leu224Phe,RC4,RC4_Mut_GDEh1026,E01,GDEhDisVCh_40054,C12,2.0,RC4_Mut_GDDh1026,E01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,1.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,GBA1_Leu224Phe,1,155238225,G,A,1
2,GBA1,ENSG00000177628,2,113,CCSBVarC000003,ALE00000003,NC_000001.11:155237453:C:T,887G>A,Arg296Gln,RC4,RC4_Mut_GDEh1026,F01,GDEhDisVCh_40054,D12,2.0,RC4_Mut_GDDh1026,F01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,7.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,GBA1_Arg296Gln,1,155237453,C,T,1
3,GBA1,ENSG00000177628,2,231,CCSBVarC000004,ALE00000004,NC_000001.11:155235252:A:G,1448T>C,Leu483Pro,RC4,RC4_Mut_GDEh1026,G01,GDEhDisVCh_40054,E12,2.0,RC4_Mut_GDDh1026,G01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,GBA1_Leu483Pro,1,155235252,A,G,1
4,GBA1,ENSG00000177628,2,213510,CCSBVarC003869,ALE00003869,NC_000001.11:155239934:G:A,259C>T,Arg87Trp,CEGS2,CegsMutGDEh1035,B03,GDEhDisVCh_40054,B02,2.0,CegsMutGDDh1035,B03,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,GBA1_Arg87Trp,1,155239934,G,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,SRY,ENSG00000184895,100070152,229149,CCSBVarC007069,ALE00007067,NC_000024.10:2787412:C:T,192G>A,Met64Ile,CEGS2,CegsMutGDEh1023,D05,GDEhDisVCh_40029,D09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,SRY_Met64Ile,24,2787412,C,T,Y
7494,SRY,ENSG00000184895,100070152,229151,CCSBVarC007072,ALE00007070,NC_000024.10:2787267:C:T,337G>A,Ala113Thr,CEGS2,CegsMutGDEh1023,F09,GDEhDisVCh_40029,H09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,SRY_Ala113Thr,24,2787267,C,T,Y
7495,SRY,ENSG00000184895,100070152,229152,CCSBVarC007068,ALE00007066,NC_000024.10:2787207:G:A,397C>T,Arg133Trp,CEGS2,CegsMutGDEh1023,E09,GDEhDisVCh_40029,G09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,SRY_Arg133Trp,24,2787207,G,A,Y
7496,CTNNB1,ENSG00000168036,100070227,212487,CCSBVarC007073,ALE00007071,NC_000003.12:41225785:A:G,860A>G,Asn287Ser,CEGS2,CegsMutGDEh1023,B07,GDEhDisVCh_40034,E09,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,CTNNB1_Asn287Ser,3,41225785,A,G,3


symbol,ensembl_gene_id,orf_id,mut_id,ccsb_mutation_id,ccsb_allele_id,spdi,nt_change,aa_change,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,mislocalization_plate,mislocalization_well,entry_sequenced,entry_sequence_confirmation_class,db_sequenced,db_sequence_confirmation_class,n2h_sequenced,n2h_sequence_confirmation_class,n2h_ref_sequence_confirmation_class,dualip_sequenced,dualip_sequence_confirmation_class,dualip_ref_sequence_confirmation_class,mislocalization_sequenced,mislocalization_sequence_confirmation_class,mislocalization_ref_sequence_confirmation_class,gene_variant,chr_num,nuc_loc,ref_allele,alt_allele,Chrom
str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,i64,f64,i64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,str,i64,i64,str,str,str
"""GBA1""","""ENSG00000177628""",2,6,"""CCSBVarC000001""","""ALE0000584""","""NC_000001.11:155240033:C:G""","""160G>C""","""Val54Leu""","""RC4""","""RC4_Mut_GDEh1026""","""H01""","""GDEhDisVCh_40054""","""F12""",2.0,"""RC4_Mut_GDDh1026""","""H01""",null,null,null,null,null,null,1,1.0,1,2.0,0,null,null,0,null,null,0,null,null,"""GBA1_Val54Leu""",1,155240033,"""C""","""G""","""1"""
"""GBA1""","""ENSG00000177628""",2,73,"""CCSBVarC000002""","""ALE00000002""","""NC_000001.11:155238225:G:A""","""670C>T""","""Leu224Phe""","""RC4""","""RC4_Mut_GDEh1026""","""E01""","""GDEhDisVCh_40054""","""C12""",2.0,"""RC4_Mut_GDDh1026""","""E01""",null,null,null,null,null,null,1,1.0,1,1.0,0,null,null,0,null,null,0,null,null,"""GBA1_Leu224Phe""",1,155238225,"""G""","""A""","""1"""
"""GBA1""","""ENSG00000177628""",2,113,"""CCSBVarC000003""","""ALE00000003""","""NC_000001.11:155237453:C:T""","""887G>A""","""Arg296Gln""","""RC4""","""RC4_Mut_GDEh1026""","""F01""","""GDEhDisVCh_40054""","""D12""",2.0,"""RC4_Mut_GDDh1026""","""F01""",null,null,null,null,null,null,1,1.0,1,7.0,0,null,null,0,null,null,0,null,null,"""GBA1_Arg296Gln""",1,155237453,"""C""","""T""","""1"""
"""GBA1""","""ENSG00000177628""",2,231,"""CCSBVarC000004""","""ALE00000004""","""NC_000001.11:155235252:A:G""","""1448T>C""","""Leu483Pro""","""RC4""","""RC4_Mut_GDEh1026""","""G01""","""GDEhDisVCh_40054""","""E12""",2.0,"""RC4_Mut_GDDh1026""","""G01""",null,null,null,null,null,null,1,1.0,1,2.0,0,null,null,0,null,null,0,null,null,"""GBA1_Leu483Pro""",1,155235252,"""A""","""G""","""1"""
"""GBA1""","""ENSG00000177628""",2,213510,"""CCSBVarC003869""","""ALE00003869""","""NC_000001.11:155239934:G:A""","""259C>T""","""Arg87Trp""","""CEGS2""","""CegsMutGDEh1035""","""B03""","""GDEhDisVCh_40054""","""B02""",2.0,"""CegsMutGDDh1035""","""B03""",null,null,null,null,null,null,1,1.0,1,2.0,0,null,null,0,null,null,0,null,null,"""GBA1_Arg87Trp""",1,155239934,"""G""","""A""","""1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SRY""","""ENSG00000184895""",100070152,229149,"""CCSBVarC007069""","""ALE00007067""","""NC_000024.10:2787412:C:T""","""192G>A""","""Met64Ile""","""CEGS2""","""CegsMutGDEh1023""","""D05""","""GDEhDisVCh_40029""","""D09""",1.0,null,null,null,null,null,null,null,null,1,1.0,0,null,0,null,null,0,null,null,0,null,null,"""SRY_Met64Ile""",24,2787412,"""C""","""T""","""Y"""
"""SRY""","""ENSG00000184895""",100070152,229151,"""CCSBVarC007072""","""ALE00007070""","""NC_000024.10:2787267:C:T""","""337G>A""","""Ala113Thr""","""CEGS2""","""CegsMutGDEh1023""","""F09""","""GDEhDisVCh_40029""","""H09""",1.0,null,null,null,null,null,null,null,null,1,1.0,0,null,0,null,null,0,null,null,0,null,null,"""SRY_Ala113Thr""",24,2787267,"""C""","""T""","""Y"""
"""SRY""","""ENSG00000184895""",100070152,229152,"""CCSBVarC007068""","""ALE00007066""","""NC_000024.10:2787207:G:A""","""397C>T""","""Arg133Trp""","""CEGS2""","""CegsMutGDEh1023""","""E09""","""GDEhDisVCh_40029""","""G09""",1.0,null,null,null,null,null,null,null,null,1,1.0,0,null,0,null,null,0,null,null,0,null,null,"""SRY_Arg133Trp

In [6]:
pdest_layout_seq_conf_df = pdest_layout_df.join(
    seq_confirm_res_df, 
    on=["orf_id","mut_id"],
    how="left"
)
print(pdest_layout_seq_conf_df["dualip_sequence_confirmation_class"].unique())

pdest_layout_seq_conf_df = pdest_layout_seq_conf_df.with_columns(
    pl.col("aa_change").fill_null("WT"),
    pl.col("nt_change").fill_null("WT")
)

pdest_layout_seq_conf_df.write_csv(f'{dualipa_outputs}/dualipa_experimental_layout.csv')

shape: (9,)
Series: 'dualip_sequence_confirmation_class' [f64]
[
	null
	1.0
	2.0
	3.0
	4.0
	5.0
	6.0
	7.0
	99.0
]


In [8]:
# Get allele metadata
# allele_info_df = pd.read_table(f"{allele_collection_inputs}/VarChamp_Consolidated_allele_collection.tsv")
# other_cols = [i for i in allele_info_df.columns if "mutation_id_" not in i]
# mut_cols = [i for i in allele_info_df.columns if "mutation_id_" in i]
# allele_info_df = allele_info_df.melt(id_vars=other_cols, 
#                                      value_vars=mut_cols, 
#                                      var_name='mut_type', 
#                                      value_name='mut_id')
# allele_info_df
# Get gene-level metadata
# gene_info_df = pd.read_csv(f"{allele_collection_outputs}/orf_info.csv")
# gene_info_df = gene_info_df[['orf_id', 'symbol']]
# orf_to_symbol = gene_info_df.set_index("orf_id")["symbol"].to_dict()
# orf_to_symbol = {str(k): v for k, v in orf_to_symbol.items()}

In [9]:
# print(pdest_layout_seq_conf_df["orf_id"].unique())

## Pandas version:
## QC with allele collection df for consistency
## Skipped for future
# meta_cols = ['orf_id', 'mut_id', 'aa_change', 'nt_change', 'symbol']
# # pdest_layout_seq_conf_df = pdest_layout_seq_conf_df.merge(allele_info_df[meta_cols], on=['orf_id', 'mut_id'], how='left', suffixes=("", "_ac"))
# # pdest_layout_seq_conf_df
# # assert(all(pdest_layout_seq_conf_df.dropna(subset="symbol")["symbol"]==pdest_layout_seq_conf_df.dropna(subset="symbol")["symbol_ac"]))
# # assert(all(pdest_layout_seq_conf_df.dropna(subset="aa_change")["aa_change"]==pdest_layout_seq_conf_df.dropna(subset="aa_change")["aa_change_ac"]))
# # assert(all(pdest_layout_seq_conf_df.dropna(subset="nt_change")["nt_change"]==pdest_layout_seq_conf_df.dropna(subset="nt_change")["nt_change_ac"]))
# # pdest_layout_seq_conf_df = pdest_layout_seq_conf_df.drop(columns=["symbol_ac","aa_change_ac","nt_change_ac"])

# ## replace NAs
# pdest_layout_seq_conf_df["orf_id"] = pdest_layout_seq_conf_df["orf_id"].astype("Int64").astype(str)
# pdest_layout_seq_conf_df["orf_id"] = pdest_layout_seq_conf_df["orf_id"].replace("<NA>", np.nan)
# pdest_layout_seq_conf_df

# ## Replace missing symbols
# pdest_layout_seq_conf_df[["aa_change", "nt_change"]] = pdest_layout_seq_conf_df[["aa_change", "nt_change"]].fillna("WT")
# pdest_layout_seq_conf_df["symbol"] = pdest_layout_seq_conf_df["symbol"].fillna(pdest_layout_seq_conf_df["orf_id"].map(orf_to_symbol))
# pdest_layout_seq_conf_df.to_csv(f'{dualipa_outputs}/dualipa_experimental_layout.csv', index=None)